## Library

In [ ]:
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import os
import torch
from langchain.vectorstores import FAISS
import pandas as pd
from tqdm import tqdm
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import List, Optional, Callable
from typing_extensions import TypedDict, Annotated
from langgraph.graph import END, StateGraph, START
from langchain_core.runnables import RunnableConfig
from langgraph.graph.state import CompiledStateGraph

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
)


# API KEY 정보로드
load_dotenv()

In [ ]:
seed = 1
emb = "koe5"
retrieve_k = 3

## Data

In [ ]:
validation = pd.read_csv(f"민원데이터/validation_민원_{seed}.csv")
test = pd.read_csv(f"민원데이터/test_민원_{seed}.csv")

## Vectorstore

In [ ]:
device = torch.device("cpu")


class KUREEmbedding:
    def __init__(self, model_name="nlpai-lab/KURE-v1"):
        self.model = SentenceTransformer(model_name, trust_remote_code=True).to(device)

    def embed_documents(self, texts):
        embeddings = self.model.encode(texts, convert_to_numpy=True)
        return embeddings

    def embed_query(self, text):
        return self.embed_documents([text])[0]


class KoE5Embedding(KUREEmbedding):
    def __init__(self, model_name="nlpai-lab/KoE5"):
        super().__init__(model_name)

In [ ]:
vectorstore_path = f"embedding_comparison/faiss_index_kure-v1_seed{seed}_민원"
if os.path.exists(vectorstore_path):
    embeddings = KoE5Embedding()

    vectorstore = FAISS.load_local(
        vectorstore_path,
        embeddings.embed_query,
        allow_dangerous_deserialization=True,
    )

    retriever = vectorstore.as_retriever(search_kwargs={"k": retrieve_k})

## Classifier

In [ ]:
class DocumentsClassification(BaseModel):
    prediction: str = Field(
        description="Predicted classification label for the statement. Possible labels: 요청/개선, 문의(질의), 건의/제기, 항의, 고충/토로, 협조, 감사."
    )

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
structured_llm_grader = llm.with_structured_output(DocumentsClassification)

## Prompt & Chain

In [ ]:
classification_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Classify the query into one of: 요청/개선, 문의(질의), 건의/제기, 항의, 고충/토로, 협조, 감사.\n"
            'Return in JSON: {{"prediction": "category"}}',
        ),
        (
            "human",
            "reference document: \n\n {documents} \n\n Input document: {question}",
        ),
    ]
)

classification_grader = classification_prompt | structured_llm_grader

## Heuristic Filtering

In [ ]:
class GradeDocuments(BaseModel):

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

structured_llm_grader = llm.with_structured_output(GradeDocuments)

## Heuristic Filtering Prompt & Chain

In [ ]:
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)
retrieval_grader = grade_prompt | structured_llm_grader

## Summary

In [ ]:
system = """
    - Label 값이 건의/제기이면, 해당 민원 내용은 건의/제기로 분류되어야 한다고 요약하세요.
    - Label 값이 고충/토로이면, 해당 민원 내용은 고충/토로로 분류되어야 한다고 요약하세요.
    - Label 값이 요청/개선이면, 해당 민원 내용은 요청/개선으로 분류되어야 한다고 요약하세요.
    - Label 값이 협조이면, 해당 민원 내용은 협조로 분류되어야 한다고 요약하세요.
    - Label 값이 문의(질의)이면, 해당 민원 내용은 문의(질의)로 분류되어야 한다고 요약하세요.
    - Label 값이 감사이면, 해당 민원 내용은 감사로 분류되어야 한다고 요약하세요.
    - Label 값이 항의이면, 해당 민원 내용은 항의로 분류되어야 한다고 요약하세요.
    다음 문장들을 위 기준에 맞춰 요약하세요.
     """

summary_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {documents} \n Formulate an improved question.",
        ),
    ]
)

summary_rewriter = summary_prompt | llm | StrOutputParser()

## LangGraph State

In [ ]:
class GraphState(TypedDict):
    question: Annotated[str, "Question"]
    generation: Annotated[str, "LLM Generation"]
    documents: Annotated[Optional[List[str]], "Retrieved Documents"]
    prediction: Annotated[str, "prediction result"]

## Node

In [ ]:
def retrieve(state):
    question = state["question"]
    documents = retriever.invoke(question)
    return {"documents": documents}


def generate(state):
    question = state["question"]
    documents = state["documents"]
    generation = classification_grader.invoke(
        {"question": question, "documents": documents}
    )
    prediction = generation.prediction
    return {"generation": generation, "prediction": prediction}


def grade_documents(state):
    question = state["question"]
    documents = state["documents"]
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            filtered_docs.append(d)
        else:
            continue
    return {"documents": filtered_docs}


def summary(state):
    documents = state["documents"]
    doc_summary = summary_rewriter.invoke({"documents": documents})
    return {"documents": doc_summary}

## Conditional Edge

In [ ]:
def decide_to_generate(state):
    state["question"]
    filtered_documents = state["documents"]
    if not filtered_documents:
        return "not relevant"
    else:
        return "relevant"

## Graph Node Generation

In [ ]:
workflow = StateGraph(GraphState)

workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("generate", generate)  # generatae
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("summary", summary)

workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_edge("summary", "generate")

workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "relevant": "summary",
        "not relevant": "generate",
    },
)

workflow.add_edge("generate", END)

app = workflow.compile()

In [ ]:
def invoke_graph_to_dataframe(
    graph: CompiledStateGraph,
    inputs: dict,
    config: RunnableConfig,
    node_names: List[str] = [],
    callback: Callable = None,
):
    result_data = []

    def format_namespace(namespace):
        return namespace[-1].split(":")[0] if len(namespace) > 0 else "root graph"

    for namespace, chunk in graph.stream(
        inputs, config, stream_mode="updates", subgraphs=True
    ):
        for node_name, node_chunk in chunk.items():
            if len(node_names) > 0 and node_name not in node_names:
                continue
            if callback is not None:
                callback({"node": node_name, "content": node_chunk})
            formatted_namespace = format_namespace(namespace)

            if isinstance(node_chunk, dict):
                for k, v in node_chunk.items():
                    result_data.append(
                        {
                            "namespace": formatted_namespace,
                            "node_name": node_name,
                            "key": k,
                            "value": v if not isinstance(v, list) else str(v),
                        }
                    )
            else:
                if node_chunk is not None:
                    for item in node_chunk:
                        result_data.append(
                            {
                                "namespace": formatted_namespace,
                                "node_name": node_name,
                                "key": None,
                                "value": item,
                            }
                        )
    df = pd.DataFrame(result_data)
    result_df = pd.DataFrame(
        columns=[
            "prediction",
        ]
    )

    result_df["prediction"] = df[df["key"] == "prediction"]["value"].values
    return result_df

## Prediction

In [ ]:
results = []
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": 1234})

for idx, row in tqdm(test.iterrows(), total=len(test), desc="Processing"):
    question = row["Q_refined"]
    answer = row["predication"]
    inputs = {"question": question}
    df_results = invoke_graph_to_dataframe(
        app,
        inputs,
        config,
    )
    df_results["true"] = answer
    results.append(df_results)

    df_results = pd.concat(results, ignore_index=True)

df_results["prediction_label"] = df_results["prediction_label"].replace(
    {"고충": "고충/토로", "토로": "고충/토로", "토로/고충": "고충/토로"}
)
df_results["prediction_label"] = df_results["prediction_label"].replace(
    {"요청": "요청/개선", "개선": "요청/개선", "개선/요청": "요청/개선"}
)
df_results["prediction_label"] = df_results["prediction_label"].replace(
    {"건의": "건의/제기", "제기": "건의/제기", "제기/건의": "건의/제기"}
)
df_results["prediction_label"] = df_results["prediction_label"].replace(
    {"문의": "문의(질의)", "질의": "문의(질의)", "질의(문의)": "문의(질의)"}
)

## Evaluation

In [ ]:
y_true = df_results["actual_label"]
y_pred = df_results["prediction_label"]

accuracy = accuracy_score(y_true, y_pred)
precision_macro = precision_score(y_true, y_pred, average="macro")
recall_macro = recall_score(y_true, y_pred, average="macro")
f1_macro = f1_score(y_true, y_pred, average="macro")
f1_weighted = f1_score(y_true, y_pred, average="weighted")

conf_matrix = confusion_matrix(y_true, y_pred)
classification_rep = classification_report(y_true, y_pred, output_dict=True)

print("\n===== Classification Performance Results =====")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (Macro): {precision_macro:.4f}")
print(f"Recall (Macro): {recall_macro:.4f}")
print(f"F1-score (Macro): {f1_macro:.4f}")
print(f"F1-score (Weighted): {f1_weighted:.4f}")

print("\n===== Classification Confusion Matrix =====")
print(conf_matrix)

print("\n===== Detailed Classification Report =====")
print(classification_report(y_true, y_pred))